# $$ Práctica\ Final $$

##### $$Construcción\ de\ base\ de\ datos\ de\ análisis.$$
##### $$ Mariana\ Lugo\ Ibarra\ y\ Miguel\ Ángel\ Reyes\ Retana$$

In [29]:
options(repr.plot.width=8, repr.plot.height=8)
library(readxl)
library(ggplot2)
library(dplyr)
library(tidyverse)
library(stringr)

## Hipotesis de Investigación:

La motivación de la investigación resulta del trabajo realizado por el economista Raj Chetty en Estados Unidos donde se observa que existe una relación negativa entre el ingreso promedio de un barrio y el número de casos confirmados con SARS-COVID-19. 

El trabajo pretende investigar si existe una relación negativa entre defunciones, casos confirmados con SARS-COVID-19 y el ingreso promedio de un municipio.  

## Datos:

Para la investigación se obtienen los datos públicos del COVID-19 en México, datos de pobreza del CONEVAL, datos de incidencia delictiva del fuero común del Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública (SESNSP), y datos privados y anonimizados de precios promedio de viviendas financiadas por Infonavit y el salario diario integrado promedio de acreditados del Infonavit.

El nivel de análisis es municipal.

A continuación se presentan los datos de las distintas fuentes:

$COVID-19$: 
* Se utilizan los datos diarios de los casos confirmados y defunciones reportados en https://coronavirus.gob.mx/datos/ por municipio. El corte de información es al 17 de julio de 2020.

In [30]:
confirmados <- read.csv ("Datos/COVID19/Casos_Diarios_Municipio_Confirmados_20200717.csv")
defunciones<-read.csv ("Datos/COVID19/Defunciones_20200717.csv")

In [31]:
head(confirmados)
dim(confirmados)
head(defunciones)
dim(defunciones)

cve_ent,poblacion,nombre,X12.01.2020,X13.01.2020,X14.01.2020,X15.01.2020,X16.01.2020,X17.01.2020,X18.01.2020,...,X08.07.2020,X09.07.2020,X10.07.2020,X11.07.2020,X12.07.2020,X13.07.2020,X14.07.2020,X15.07.2020,X16.07.2020,X17.07.2020
16046,15290,Juarez,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16047,22358,Jungapeo,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
16048,5862,Lagunillas,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16049,18769,Madero,0,0,0,0,0,0,0,...,0,1,0,0,0,2,0,0,0,0
5035,744247,Torreon,0,0,0,0,0,0,0,...,53,67,66,12,6,21,22,22,4,0
5036,23781,Viesca,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


[1] 2457  191

cve_ent,poblacion,nombre,X17.03.2020,X18.03.2020,X19.03.2020,X20.03.2020,X21.03.2020,X22.03.2020,X23.03.2020,...,X08.07.2020,X09.07.2020,X10.07.2020,X11.07.2020,X12.07.2020,X13.07.2020,X14.07.2020,X15.07.2020,X16.07.2020,X17.07.2020
16046,15290,Juarez,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16047,22358,Jungapeo,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16048,5862,Lagunillas,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16049,18769,Madero,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5035,744247,Torreon,0,0,0,0,0,0,0,...,3,2,3,2,0,2,2,1,0,0
5036,23781,Viesca,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


[1] 2457  126

Para obtener el total de casos confirmados y defunciones por municipio se suma el número de casos diarios confirmado y defunciones al 17 de julio de 2020, de acuerdo con las dimensiones de los sets de datos respectivos.

In [32]:
confirmados$total<-rowSums(confirmados[,c(4:191)])
defunciones$total<-rowSums(defunciones[,c(4:126)])

$CONEVAL$: 

Se agregan datos de pobreza para el año 2015. Los datos viene desagregados a nivel municipal y toman en cuenta los siguientes indicadores (carencias sociales) para la medición de la pobreza multidimensional: Ingreso corriente per cápita; rezago educativo; acceso a servicios de salud; acceso a seguridad social; calidad y espacios de la vivienda; acceso a servicios básicos de la vivienda; acceso a la alimentación; y grado de cohesión social. Para que una persona se considere en situación de pobreza, debe tener al menos tres carencias sociales y un ingreso por debajo de la línea de bienestar. Para este trabajo se considera el porcentaje y el número de la población en situación de pobreza para el año 2015. 

In [33]:
coneval<-read.csv("Datos/CONEVAL/Concentrado_pobreza_municipios_2015.csv")
coneval<-coneval[,c("Entidad","Nombre_ent", "cve_mun", "pob_porc_2015","pob_per_2015")]
coneval$cve_mun<- str_pad(string=as.character(coneval$cve_mun),side='left',width=5, pad="0")
names(coneval)[3]<-"CV_MUN"
head(coneval)

Entidad,Nombre_ent,CV_MUN,pob_porc_2015,pob_per_2015
01,Aguascalientes,01001,26.1,"224,949"
01,Aguascalientes,01002,54.0,"25,169"
01,Aguascalientes,01003,56.8,"29,951"
01,Aguascalientes,01004,43.2,"6,620"
01,Aguascalientes,01005,25.0,"31,479"
01,Aguascalientes,01006,34.9,"15,972"


$Secretariado\ Ejecutivo\ del\ Sistema\ Nacional\ de\ Seguridad\ Pública\ (SESNSP):$

Se descargaron los datos de Incidencia Delictiva del Fuero Común a nivel municipal con el número de delitos por mes del año 2015 a mayo del 2020 (último periodo disponible).

Se considera al los homicidios y al robo de vehículo como los delitos con menos cifra negra y, por lo tanto, los delitos asertivos de violencia.

Se construyen las tasas de homicidios y robo a vehículo por cada cien mil habitantes por año. Con el objetivo de tener una tasa del delito, y por lo tanto violencia, más robusta, se calcula una tasa promedio del año 2015 a mayo 2020.

In [34]:
d2015<-read_excel("Datos/SEGURIDAD/2015.xlsx")
d2016<-read_excel("Datos/SEGURIDAD/2016.xlsx")
d2017<-read_excel("Datos/SEGURIDAD/2017.xlsx")
d2018<-read_excel("Datos/SEGURIDAD/2018.xlsx")
d2019<-read_excel("Datos/SEGURIDAD/2019.xlsx")
d052020<-read_excel("Datos/SEGURIDAD/2020.xlsx")

Se construyen los delitos totales por año:

In [35]:
d2015$total<-rowSums(d2015[,c(10:21)])
d2016$total<-rowSums(d2016[,c(10:21)])
d2017$total<-rowSums(d2017[,c(10:21)])
d2018$total<-rowSums(d2018[,c(10:21)])
d2019$total<-rowSums(d2019[,c(10:21)])
d052020$total<-rowSums(d052020[,c(10:14)])

Se calcula el número de homicidios y robo a vehículo por municipio por cada año. 

In [36]:
d2015_h<-d2015%>% group_by(`Cve. Municipio`) %>% filter(`Tipo de delito`=="Homicidio") %>% summarise (total_h=sum(total))
d2016_h<-d2016%>% group_by(`Cve. Municipio`) %>% filter(`Tipo de delito`=="Homicidio") %>% summarise (total_h=sum(total))
d2017_h<-d2017%>% group_by(`Cve. Municipio`) %>% filter(`Tipo de delito`=="Homicidio") %>% summarise (total_h=sum(total))
d2018_h<-d2018%>% group_by(`Cve. Municipio`) %>% filter(`Tipo de delito`=="Homicidio") %>% summarise (total_h=sum(total))
d2019_h<-d2019%>% group_by(`Cve. Municipio`) %>% filter(`Tipo de delito`=="Homicidio") %>% summarise (total_h=sum(total))
d052020_h<-d052020%>% group_by(`Cve. Municipio`) %>% filter(`Tipo de delito`=="Homicidio") %>% summarise (total_h=sum(total))

`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)


In [37]:
d2015_r<-d2015%>% group_by(`Cve. Municipio`) %>% filter(`Subtipo de delito`=="Robo de vehículo automotor") %>% summarise (total_r=sum(total))
d2016_r<-d2016%>% group_by(`Cve. Municipio`) %>% filter(`Subtipo de delito`=="Robo de vehículo automotor") %>% summarise (total_r=sum(total))
d2017_r<-d2017%>% group_by(`Cve. Municipio`) %>% filter(`Subtipo de delito`=="Robo de vehículo automotor") %>% summarise (total_r=sum(total))
d2018_r<-d2018%>% group_by(`Cve. Municipio`) %>% filter(`Subtipo de delito`=="Robo de vehículo automotor") %>% summarise (total_r=sum(total))
d2019_r<-d2019%>% group_by(`Cve. Municipio`) %>% filter(`Subtipo de delito`=="Robo de vehículo automotor") %>% summarise (total_r=sum(total))
d052020_r<-d052020%>% group_by(`Cve. Municipio`) %>% filter(`Subtipo de delito`=="Robo de vehículo automotor") %>% summarise (total_r=sum(total))

`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)


$Infonavit$: 

* Se utiliza la información del precio promedio de las viviendas a nivel municipal y salario diario integrado promedio de los acreditados por municipio con el objetivo de establecer municipios "ricos" y "pobres" y su relación con la información del COVID-19. 
* Los set de datos contienen la clave del muncipio donde se financió vivienda, el precio promedio por municipio, el salario diario integrado de los acreditados promedio por municipio, la proporción de créditos originados por año en cada municipio y el año de originación.*

In [38]:
INF10<-read_excel("Datos/INFONAVIT/INFV2010.xlsx")
INF11<-read_excel("Datos/INFONAVIT/INFV2011.xlsx")
INF12<-read_excel("Datos/INFONAVIT/INFV2012.xlsx")
INF13<-read_excel("Datos/INFONAVIT/INFV2013.xlsx")
INF14<-read_excel("Datos/INFONAVIT/INFV2014.xlsx")
INF15<-read_excel("Datos/INFONAVIT/INFV2015.xlsx")
INF16<-read_excel("Datos/INFONAVIT/INFV2016.xlsx")
INF17<-read_excel("Datos/INFONAVIT/INFV2017.xlsx")
INF18<-read_excel("Datos/INFONAVIT/INFV2018.xlsx")
INF19<-read_excel("Datos/INFONAVIT/INFV2019.xlsx")
INF0620<-read_excel("Datos/INFONAVIT/INFV062020.xlsx")

In [39]:
infonavit<-rbind(INF10,INF11, INF12,INF13,INF14,INF15,INF16,INF17,INF18,INF19,INF0620)
head(infonavit)

CV_MUN,PRECIO_PROM,SDI_PROM,PC_CRED,YEAR
01001,332299.2,4.852961,1.220829e-02,2010
01002,308960.0,6.135259,2.159995e-05,2010
01003,244566.1,2.902301,6.911985e-05,2010
01004,353464.8,4.365375,4.967989e-05,2010
01005,422150.7,6.617866,2.246395e-03,2010
01006,284349.8,4.151593,1.360797e-04,2010


In [40]:
df_inf<-infonavit %>% group_by(CV_MUN) %>% summarise(precio_mean=mean(PRECIO_PROM),
                                                    salario_mean=mean(SDI_PROM))
head(df_inf)

`summarise()` ungrouping output (override with `.groups` argument)


CV_MUN,precio_mean,salario_mean
01001,450371.7,4.767035
01002,351536.1,3.952249
01003,418039.4,3.360099
01004,368355.9,3.782037
01005,672823.9,7.371576
01006,388481.6,4.334164


Del 2010 al primer semestre del 2020 el Infonavit ha financiado viviendas en 1517 municipios.

In [41]:
dim(df_inf)

[1] 1517    3

**Se construye la base de datos para análisis:**

Se homologa las claves de municipio y nombre del campo para todos los sets de datos:

In [42]:
confirmados$cve_ent<- str_pad(string=as.character(confirmados$cve_ent),side='left',width=5, pad="0")
defunciones$cve_ent<- str_pad(string=as.character(defunciones$cve_ent),side='left',width=5, pad="0")

In [43]:
d2015_h$`Cve. Municipio`<-str_pad(string=as.character(d2015_h$`Cve. Municipio`),side='left',width=5, pad="0")
d2016_h$`Cve. Municipio`<-str_pad(string=as.character(d2016_h$`Cve. Municipio`),side='left',width=5, pad="0")
d2017_h$`Cve. Municipio`<-str_pad(string=as.character(d2017_h$`Cve. Municipio`),side='left',width=5, pad="0")
d2018_h$`Cve. Municipio`<-str_pad(string=as.character(d2018_h$`Cve. Municipio`),side='left',width=5, pad="0")
d2019_h$`Cve. Municipio`<-str_pad(string=as.character(d2019_h$`Cve. Municipio`),side='left',width=5, pad="0")
d052020_h$`Cve. Municipio`<-str_pad(string=as.character(d052020_h$`Cve. Municipio`),side='left',width=5, pad="0")

In [44]:
d2015_r$`Cve. Municipio`<-str_pad(string=as.character(d2015_r$`Cve. Municipio`),side='left',width=5, pad="0")
d2016_r$`Cve. Municipio`<-str_pad(string=as.character(d2016_r$`Cve. Municipio`),side='left',width=5, pad="0")
d2017_r$`Cve. Municipio`<-str_pad(string=as.character(d2017_r$`Cve. Municipio`),side='left',width=5, pad="0")
d2018_r$`Cve. Municipio`<-str_pad(string=as.character(d2018_r$`Cve. Municipio`),side='left',width=5, pad="0")
d2019_r$`Cve. Municipio`<-str_pad(string=as.character(d2019_r$`Cve. Municipio`),side='left',width=5, pad="0")
d052020_r$`Cve. Municipio`<-str_pad(string=as.character(d052020_r$`Cve. Municipio`),side='left',width=5, pad="0")

In [45]:
names(confirmados)[1]<-"CV_MUN"
names(defunciones)[1]<-"CV_MUN"
names(d2015_h)[1]<-"CV_MUN"
names(d2016_h)[1]<-"CV_MUN"
names(d2017_h)[1]<-"CV_MUN"
names(d2018_h)[1]<-"CV_MUN"
names(d2019_h)[1]<-"CV_MUN"
names(d052020_h)[1]<-"CV_MUN"
names(d2015_r)[1]<-"CV_MUN"
names(d2016_r)[1]<-"CV_MUN"
names(d2017_r)[1]<-"CV_MUN"
names(d2018_r)[1]<-"CV_MUN"
names(d2019_r)[1]<-"CV_MUN"
names(d052020_r)[1]<-"CV_MUN"

Nos quedamos solo con los casos confirmados totales y defunciones totales por municipio:

In [46]:
confirmados<-confirmados[,c(1:3,192)]
defunciones<-defunciones[,c(1,127)]

In [47]:
datos<-left_join(df_inf, confirmados, by="CV_MUN")
datos<-left_join(datos, defunciones, by="CV_MUN")

In [48]:
names(datos)[6]<-"confirmados_total"
names(datos)[7]<-"defunciones_total"
head(datos)
dim(datos)

CV_MUN,precio_mean,salario_mean,poblacion,nombre,confirmados_total,defunciones_total
01001,450371.7,4.767035,961977,Aguascalientes,2465,173
01002,351536.1,3.952249,50864,Asientos,48,5
01003,418039.4,3.360099,60760,Calvillo,84,1
01004,368355.9,3.782037,16918,Cosio,49,0
01005,672823.9,7.371576,130184,Jesus Maria,104,2
01006,388481.6,4.334164,50032,Pabellon de Arteaga,113,6


[1] 1517    7

Construimos la tasa promedio de los delitos por cada cien mil habitantes:

In [49]:
poblacion<-datos[,c(1,4)]

In [50]:
d2015_h<-left_join(poblacion, d2015_h, by="CV_MUN")
d2015_h<-d2015_h %>% mutate(ts_15_h=(total_h/poblacion)*100000)

d2016_h<-left_join(poblacion, d2016_h, by="CV_MUN")
d2016_h<-d2016_h %>% mutate(ts_16_h=(total_h/poblacion)*100000)

d2017_h<-left_join(poblacion, d2017_h, by="CV_MUN")
d2017_h<-d2017_h %>% mutate(ts_17_h=(total_h/poblacion)*100000)

d2018_h<-left_join(poblacion, d2018_h, by="CV_MUN")
d2018_h<-d2018_h %>% mutate(ts_18_h=(total_h/poblacion)*100000)

d2019_h<-left_join(poblacion, d2019_h, by="CV_MUN")
d2019_h<-d2019_h %>% mutate(ts_19_h=(total_h/poblacion)*100000)

d052020_h<-left_join(poblacion, d052020_h, by="CV_MUN")
d052020_h<-d052020_h %>% mutate(ts_20_h=(total_h/poblacion)*100000)

In [51]:
d2015_r<-left_join(poblacion, d2015_r, by="CV_MUN")
d2015_r<-d2015_r %>% mutate(ts_15_r=(total_r/poblacion)*100000)

d2016_r<-left_join(poblacion, d2016_r, by="CV_MUN")
d2016_r<-d2016_r %>% mutate(ts_16_r=(total_r/poblacion)*100000)

d2017_r<-left_join(poblacion, d2017_r, by="CV_MUN")
d2017_r<-d2017_r %>% mutate(ts_17_r=(total_r/poblacion)*100000)

d2018_r<-left_join(poblacion, d2018_r, by="CV_MUN")
d2018_r<-d2018_r %>% mutate(ts_18_r=(total_r/poblacion)*100000)

d2019_r<-left_join(poblacion, d2019_r, by="CV_MUN")
d2019_r<-d2019_r %>% mutate(ts_19_r=(total_r/poblacion)*100000)

d052020_r<-left_join(poblacion, d052020_r, by="CV_MUN")
d052020_r<-d052020_r %>% mutate(ts_20_r=(total_r/poblacion)*100000)

In [52]:
delitos<-left_join(d2015_h,d2016_h,by="CV_MUN")
delitos<-left_join(delitos,d2017_h,by="CV_MUN")
delitos<-left_join(delitos,d2018_h,by="CV_MUN")
delitos<-left_join(delitos,d2019_h,by="CV_MUN")
delitos<-left_join(delitos,d052020_h,by="CV_MUN")
delitos<-left_join(delitos,d2015_r,by="CV_MUN")
delitos<-left_join(delitos,d2016_r,by="CV_MUN")
delitos<-left_join(delitos,d2017_r,by="CV_MUN")
delitos<-left_join(delitos,d2018_r,by="CV_MUN")
delitos<-left_join(delitos,d2019_r,by="CV_MUN")
delitos<-left_join(delitos,d052020_r,by="CV_MUN")

ts_homicidios<-delitos[,c("CV_MUN","ts_15_h","ts_16_h","ts_17_h","ts_18_h","ts_19_h","ts_20_h")]
ts_homicidios$ts_promedio_h<-rowMeans(ts_homicidios[,c(2:7)])
ts_robos<-delitos[,c("CV_MUN","ts_15_r","ts_16_r","ts_17_r","ts_18_r","ts_19_r","ts_20_r")]
ts_robos$ts_promedio_r<-rowMeans(ts_robos[,c(2:7)])

ts_homicidio_promedio<-ts_homicidios[,c(1,8)]
ts_robo_promedio<-ts_robos[,c(1,8)]

In [53]:
datos<-left_join(datos, ts_homicidio_promedio, by="CV_MUN")
datos<-left_join(datos, ts_robo_promedio, by="CV_MUN")

Por último, se une la información de pobreza de CONEVAL:

In [54]:
datos<-left_join(datos, coneval, by="CV_MUN")
head(datos)

CV_MUN,precio_mean,salario_mean,poblacion,nombre,confirmados_total,defunciones_total,ts_promedio_h,ts_promedio_r,Entidad,Nombre_ent,pob_porc_2015,pob_per_2015
01001,450371.7,4.767035,961977,Aguascalientes,2465,173,13.99895,225.47317,01,Aguascalientes,26.1,"224,949"
01002,351536.1,3.952249,50864,Asientos,48,5,22.60931,35.06082,01,Aguascalientes,54.0,"25,169"
01003,418039.4,3.360099,60760,Calvillo,84,1,17.55541,58.42660,01,Aguascalientes,56.8,"29,951"
01004,368355.9,3.782037,16918,Cosio,49,0,20.68802,45.31663,01,Aguascalientes,43.2,"6,620"
01005,672823.9,7.371576,130184,Jesus Maria,104,2,16.38706,114.83746,01,Aguascalientes,25.0,"31,479"
01006,388481.6,4.334164,50032,Pabellon de Arteaga,113,6,17.32225,54.96482,01,Aguascalientes,34.9,"15,972"


In [55]:
dim(datos)

[1] 1517   13

In [56]:
write.csv(datos, "Datos/datos.csv")